# 最終課題

### やること

URLを検索

検索したURLからタイトルを取得

取得したタイトルとURLを辞書型に保存dictionary(key value)

生成したdictionaryをもとに新たにURLを検索

In [70]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
# 一部の相対URLを取得するコードを追加
# URLにリクエストを送信し、BeautifulSoupオブジェクトを返す関数を定義 
def fetch_url(url):
    time.sleep(0.3)
    try:
        res = requests.get(url)
        res.encoding = res.apparent_encoding
    except requests.exceptions.ConnectionError:
        print(f"[接続エラー] {url} に接続できません。スキップします。")
        return None
    except requests.exceptions.RequestException as e:
        print(f"[HTTPエラー] {url} ({e})")
        return None
    # ステータスコードでエラーが出た場合と、URLがHTML以外の場合の処理
    if res.status_code != 200:
        print(f"[{res.status_code}] アクセスエラー: {url}")
        return None
    if 'text/html' not in res.headers.get('Content-Type', ''):
        print(f"{res.headers.get('Content-Type', '')}なのでスキップ")
        return None
    # 正常な場合の処理
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

# URLをチェックする関数を定義
def check_url(main_domain, url, find_list, found_url):
    sep_url = url.split('/') # スラッシュでURLを分割
    # urlの最初の4文字がhttpで始まり、ドメインを含み、未発見の絶対URLか確認
    if (url[:4] == "http" and
        len(sep_url) > 2 and
        main_domain in sep_url[2] and
        url not in found_url and
        url not in find_list):
        return url
    # urlの最初の1文字が/で始まる相対URLであるか確認
    if url[:1] == "/":
        # print("相対URL発見:" + url)
        full_url = "http://www.musashino-u.ac.jp" + url
        sep_full_url = full_url.split('/')
        if (main_domain in sep_full_url[2] and
            len(sep_full_url) > 2 and
            full_url not in found_url and
            full_url not in find_list):
            return full_url
    return None

# タイトルを取得する関数を定義
def get_title(soup):
    if soup.title and soup.title.string: # タイトルタグが存在し、その中に文字列があるか確認
        return soup.title.get_text()
    return "No title"

# ページ内のリンクをリストにする関数を定義    
def collect_urls(soup, main_domain, find_list, found_url):
    href_list = soup.find_all('a', href=True)
    for href in href_list:
        url = href['href']
        # リンクをチェックする関数を呼び出し、条件を満たす場合はリストに追加
        checked_url = check_url(main_domain, url, find_list, found_url)
        if checked_url:
            find_list.append(checked_url)
            # print("追加URL:" + checked_url)
    return find_list


   
# ウェブサイトを巡回する関数を定義    
def crawl_website(access_url, main_domain):
    
    find_list = [access_url]
    found_url = set()
    link_dic = {}

    # find_listが空になるまで繰り返す
    while find_list:
        
        # 調査するURLを取得→pop(0)で先頭要素（最も古く追加されたURL）を取り出します
        current_url = find_list.pop(0)

        # 既に発見済みのURLはスキップ
        if current_url in found_url:
            continue

        # レスポンス取得
        soup = fetch_url(current_url)
        if not soup:
            found_url.add(current_url)
            continue
        
        # リンク取得
        collect_urls(soup, main_domain, find_list, found_url)

        # タイトル取得
        title = get_title(soup)

        # 取得したURLとそのタイトルを辞書型変数に追加
        link_dic[current_url] = title  

        # 調べ終わったらURLを発見済みリストに追加
        found_url.add(current_url)

        # 途中の進捗を表示
        print(f"{len(found_url)}個完了 URL: {current_url} TITLE: {get_title(soup)}")
        
    return link_dic

In [ ]:
access_url = "https://www.musashino-u.ac.jp/"
main_domain = "musashino-u.ac.jp"

print(f"START: {access_url}")

link_dic = crawl_website(access_url, main_domain)

print("--- FINISH ---")
print(f"計 {len(link_dic)} ページを取得")

print("--- 辞書型変数の内容 ---")
print(link_dic)

START: https://www.musashino-u.ac.jp/
(1個完了 URL: https://www.musashino-u.ac.jp/ TITLE: 武蔵野大学
(2個完了 URL: http://www.musashino-u.ac.jp/ TITLE: 武蔵野大学
(3個完了 URL: https://ef.musashino-u.ac.jp/donation/ TITLE: ご寄付のお願い | 学校法人武蔵野大学
(4個完了 URL: http://www.musashino-u.ac.jp/access.html TITLE: 交通アクセス | 武蔵野大学
(5個完了 URL: http://www.musashino-u.ac.jp/admission/request.html TITLE: 資料請求 | 入試情報 | 武蔵野大学
(6個完了 URL: http://www.musashino-u.ac.jp/contact.html TITLE: お問い合わせ | 武蔵野大学
(7個完了 URL: http://www.musashino-u.ac.jp/prospective-students.html TITLE: 武蔵野大学で学びたい方 | 武蔵野大学
(8個完了 URL: http://www.musashino-u.ac.jp/students.html TITLE: 在学生の方 | 武蔵野大学
(9個完了 URL: http://www.musashino-u.ac.jp/alumni.html TITLE: 卒業生の方 | 武蔵野大学
(10個完了 URL: http://www.musashino-u.ac.jp/parents.html TITLE: 保護者の方 | 武蔵野大学
(11個完了 URL: http://www.musashino-u.ac.jp/business.html TITLE: 企業・研究者の方 | 武蔵野大学
(12個完了 URL: http://www.musashino-u.ac.jp/guide/ TITLE: 大学案内 | 武蔵野大学
(13個完了 URL: http://www.musashino-u.ac.jp/guide/profile/ TITLE: 大学紹介 | 大学案内